In [ ]:
!wget -O archive.zip https://www.dropbox.com/s/5yjv2hth42wbjz3/archive.zip?dl=0
!unzip "archive.zip"

In [ ]:
!pip install tensorlayer
!pip install emot
!pip install spellchecker
!pip install pyspellchecker

import pandas as pd
import numpy as np
import string 
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.model_selection import train_test_split
from emot.emo_unicode import EMOTICONS
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from torchtext import data
import tensorflow as tf
import tensorlayer as tl
from tensorlayer.cost import cross_entropy_seq_with_mask
from tensorlayer.models import Seq2seq
from tqdm import tqdm
from sklearn.utils import shuffle
from keras.preprocessing.text import Tokenizer
nltk.download('words')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
stopwords.words('english')
%matplotlib inline
from spellchecker import SpellChecker

In [18]:
def get_response(data, x):
    """
     Replaces tweet id with its text
    """
    row = data[data.tweet_id == int(x)]
    return '' if (len(row) == 0) else row.iloc[0]['text']

df = pd.read_csv("twcs/twcs.csv")
df.sort_values(by='tweet_id', inplace=True)

support_name = "AppleSupport"

support = df[df.author_id == support_name]
support = support[~support.in_response_to_tweet_id.isna()]
support = support[['tweet_id', 'author_id', 'text', 'in_response_to_tweet_id']]
support.loc[:, 'in_response_to_tweet_id'] = support.in_response_to_tweet_id.apply(lambda x: get_response(df, x))
support.rename(columns = {'author_id':'support_name', 'text':'answer', 'in_response_to_tweet_id':'question'}, inplace = True)

support.to_csv("data/" + support_name + "-before.csv", sep='\t', index=False)

In [21]:
chat_words = {
    "afaik": "as far as i know",
    "asap": "as soon as possible",
    "atk": "at the keyboard",
    "atm": "at the moment",
    "brb": "be right back",
    "brt": "be right there",
    "btw": "by the way",
    "b4": "before",
    "cu": "see you",
    "cya": "see you",
    "faq": "frequently asked questions",
    "fc": "fingers crossed",
    "fyi": "for your information",
    "gn": "good night",
    "gr8": "great",
    "g9": "genius",
    "ic": "i see",
    "imo": "in my opinion",
    "iow": "in other words",
    "lol": "laughing out loud",
    "l8r": "later",
    "mte": "my thoughts rxactly",
    "m8": "mate",
    "nrn": "no reply necessary",
    "oic": "oh i see",
    "rofl": "rolling on the floor laughing",
    "thx": "thank you",
    "ttyl": "talk to you later",
    "u": "you",
    "u2": "you too",
    "w8": "wait",
    "imma": "i am going to",
    "2nite": "tonight"
}

contractions = {
    "ain't": "are not",
    "aren't": "are not",
    "'bout": "about",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "here's": "here is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "I would",
    "i'd've": "I would have",
    "i'll": "I will",
    "i'll've": "I will have",
    "i'm": "I am",
    "i've": "I have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "i phone": "iphone",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so is",
    "that'd": "that had",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they're": "they are",
    "they've": "they have",
    "'til": "until",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'll": "you will",
    "you're": "you are",
    "you've": "you have",
}

def remove_chat_words_and_contractions(text):
    """
      Removes chat words listed above and contractions, also listed above from text
    """
    new_text = []
    for word in text.split(' '):
        if word in chat_words.keys():
            new_text += chat_words[word].split(' ')
        if word in contractions.keys():
            new_text += contractions[word].split(' ')
        else:
            new_text.append(word)
    return ' '.join(new_text)

def remove_urls(text):
    """
      Replaces urls from dataset with url as text
    """
    return re.sub(r'https://t\.co/\w+', 'url', text)

def remove_mentions(text):
    """
      Removes user mentions from text
    """
    return re.sub(r'@[0-9A-Za-z_\-]+', '', text)

def remove_hashtags(text):
    """
      Removes hashtags
    """
    return re.sub(r'#[0-9A-Za-z_\-]+', '', text)

def remove_emojis(text):
    """
      Removes emojis from text
    """
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"
                               u"\U0001F300-\U0001F5FF"
                               u"\U0001F680-\U0001F6FF"
                               u"\U0001F1E0-\U0001F1FF"
                               u"\U00002500-\U00002BEF"
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_emoticons(text):
    """
      Removes emoticons from text
    """
    emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in EMOTICONS) + u')')
    return emoticon_pattern.sub(r'', text)

from string import digits
puncList = ["&gt","amp","%",'newlinechar',"~",".","\t","\n", "^", "_", "*", "<", ">", ";", ":", "!", "?", "/", "\\", ",", "#", "@", "$", "&", ")", "(", "\"", "]", "[", "|", "{", "}","=","-","+","\""]
isascii = lambda s: len(s) == len(s.encode())
remove_digits = str.maketrans('', '', digits)

def remove_numbers(content):
    """
      Removes numbers from text
    """
    return content.translate(remove_digits)

def isEnglish(s):
    """
      Checks if s is english word
    """
    try:
      s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
      return False
    else:
      return True

def remove_inerpunction_and_nonascii_chars(content):
    """
      Removes inerpunction and nonascii characters from content
    """
    new_content = ""
    content = content.split(" ")
    for word in content:
        word = word.strip()
        for punc in puncList:
            word = ''.join(word.split(punc))
        if word != "" and word != " " and isEnglish(word):
            new_content = new_content + " " + word
        if not isEnglish(word):
            return ""
    return new_content.strip()

def remove_spacing(text):
    """
      Removes spaces from text
    """
    return " ".join([word for word in nltk.word_tokenize(text)])

def clean(text):
    """
      Cleans text calling functions defined above
    """
    text = re.sub(r'[“”]', '"', text)
    text = re.sub(r"’", "'", text)
    text = text.lower()
    text = remove_chat_words_and_contractions(text)
    text = remove_urls(text)
    text = remove_mentions(text)
    text = remove_hashtags(text)
    text = re.sub(r'&gt;', '>', text)
    text = re.sub(r'&lt;', '<', text)
    text = remove_emojis(text)
    text = remove_emoticons(text)
    text = remove_inerpunction_and_nonascii_chars(text)
    text = remove_numbers(text)
    text = remove_spacing(text)
    return text

support.loc[:, 'question'] = support.question.apply(lambda x: clean(x))
support.loc[:, 'answer'] = support.answer.apply(lambda x: clean(x))

In [ ]:
spell = SpellChecker()

def spelling_correction(text):
    """
      Checks if words from text are spelled correctly
    """
    corrected_text = []
    misspelled_words = spell.unknown(text.split())    
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    new_text = " ".join(corrected_text)
    return new_text
      
words = set(nltk.corpus.words.words())

support.loc[:, 'question'] = support.question.apply(lambda x: spelling_correction(x))
support.loc[:, 'answer'] = support.answer.apply(lambda x: spelling_correction(x))

In [ ]:
def get_wordnet_pos(word):
    """
      Gets the word type
    """
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()

few_stopwords = ["a", "the", "an"]
nltk_stopwords = stopwords.words()

def remove_stopwords(text):
    """
      Removes english stopwords from text
    """
    return " ".join([word for word in nltk.word_tokenize(text) if not word in nltk_stopwords])

def lemmatize(text):
    """
      Reduces the words(from text) properly ensuring that the root word belongs to the language
    """
    return " ".join([lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in nltk.word_tokenize(text)])

def stopwords_and_lemmatize(text):
    text = remove_stopwords(text)
    text = lemmatize(text)
    return text

support.loc[:, 'question'] = support.question.apply(lambda x: stopwords_and_lemmatize(x))
support.loc[:, 'answer'] = support.answer.apply(lambda x: stopwords_and_lemmatize(x))

In [ ]:
support = support[~(support.answer == '')]
support = support[~(support.question == '')]
support = support[~(support.answer == '.')]
support = support[~(support.question == '.')]
support = support[support["answer"].str.count(" ") < 25]
support = support[support["question"].str.count(" ") < 25]
#support = support[support["question"].str.count(" ") < support["answer"].str.count(" ") * 1.5]
#support = support[support["answer"].str.count(" ") < support["question"].str.count(" ") * 1.5]

train_set, rest = train_test_split(support, test_size=0.1, random_state=287)
val_set, test_set = train_test_split(rest, test_size=0.9, random_state=287)

train_set.to_csv('data/' + support_name + '-train.csv', index=False)
val_set.to_csv('data/' + support_name + '-val.csv', index=False)
test_set.to_csv('data/' + support_name + '-test.csv', index=False)

In [57]:
BATCH_SIZE = 32
EPOCHS_NUMBER = 40
VOCABULARY_SIZE = 20000
EMBEDDING_SIZE = 1024
DECODER_SEQ_LENGTH = 25 #amazon 20, uber 21
N_LAYER = 3
N_UNITS = 256
LEARNING_RATE = 0.001

In [38]:
answer_field = data.Field(init_token='<start>', eos_token='<end>', pad_token='<pad>', tokenize='spacy', lower=True, fix_length=DECODER_SEQ_LENGTH)
question_field = data.Field(init_token='<start>', eos_token='<end>', pad_token='<pad>', tokenize='spacy', lower=True, pad_first=True)
train, val, test = data.TabularDataset.splits(
    path='data',
    format='csv',
    train=support_name + '-train.csv',
    validation=support_name + '-val.csv',
    test=support_name + '-test.csv',
    fields=[
        ('tweet_id', None),
        ('support_name', None),
        ('answer', answer_field),
        ('question', question_field)
    ],
    skip_header=True
)

answer_field.build_vocab(train, min_freq=2, max_size=VOCABULARY_SIZE)
question_field.build_vocab(train, min_freq=2, max_size=VOCABULARY_SIZE)
answer_field.vocab.stoi['<unk>'] = 1
answer_field.vocab.stoi['<pad>'] = 0
question_field.vocab.stoi['<unk>'] = 1
question_field.vocab.stoi['<pad>'] = 0

train_iter, val_iter, test_iter = data.BucketIterator.splits(
    (train, val, test), batch_size=BATCH_SIZE, sort_key=lambda x: len(x.question), repeat=False, shuffle=True)

vocab = answer_field.vocab

In [ ]:
model = Seq2seq(
    decoder_seq_length=DECODER_SEQ_LENGTH,
    cell_enc=tf.keras.layers.LSTMCell,
    cell_dec=tf.keras.layers.LSTMCell,
    n_layer=N_LAYER,
    n_units=N_UNITS,
    embedding_layer=tl.layers.Embedding(vocabulary_size=VOCABULARY_SIZE, embedding_size=EMBEDDING_SIZE),
)

def predict(query):
    """
      Predicts the answer to query
    """
    with open('data/vocab' + support_name + '.pkl', 'rb') as f:
        vocab = pickle.load(f)
    with open('data/invertedVocab' + support_name + '.pkl', 'rb') as f:
        inverted_vocab = pickle.load(f)
    model.eval()
    query = clean(query)
    new_query = query
    for el in query.split():
        if el not in inverted_vocab.keys():
            new_query = ''.join(new_query.split(el))
    query = new_query
    query = ' '.join(query.split())
    if query != "":
        query_tokenized = [inverted_vocab.get(w) for w in query.split()]
        answer_tokenized = model(inputs=[[query_tokenized]], seq_length=DECODER_SEQ_LENGTH, start_token=VOCABULARY_SIZE + 1, top_n=1)
        answer = []
        for word_token in answer_tokenized[0]:
            w = vocab[word_token.numpy()]
            if w == '<end>':
                break
            answer = answer + [w]
        return " ".join(answer)
    return ""

def train():
    """
      Training the network
    """
    optimizer = tf.optimizers.Adam(learning_rate=LEARNING_RATE)
    model.train()
    for epoch in range(EPOCHS_NUMBER):
        total_loss, n_iter = 0, 0
        model.train()
        for batch in tqdm(train_iter, total=train_iter.iterations, desc='Epoch[{}/{}]'.format(epoch + 1, EPOCHS_NUMBER), leave=False):
            questions = np.array(batch.question).transpose().tolist()
            answers = np.array(batch.answer).transpose().tolist()
            target_mask = tl.prepro.sequences_get_mask(answers)
            with tf.GradientTape() as tape:
                output = model(inputs = [questions, answers])
                output = tf.reshape(output, [-1, VOCABULARY_SIZE])
                loss = cross_entropy_seq_with_mask(logits=output, target_seqs=answers, input_mask=target_mask)
                grad = tape.gradient(loss, model.all_weights)
                optimizer.apply_gradients(zip(grad, model.all_weights))
            
            total_loss += loss
            n_iter += 1

        print('Epoch [{}/{}]: loss {:.4f}'.format(epoch + 1, EPOCHS_NUMBER, total_loss / n_iter))
        tl.files.save_npz(model.all_weights, name='data/model.npz')

train()

In [ ]:
weights = tl.files.load_npz(name='data/model' + support_name + '.npz')
tl.files.assign_weights(weights, train_model)

while True:
    query = input(">> ")
    sentence = predict(query)
    print("Support >>", ' '.join(sentence))

In [ ]:
weights = tl.files.load_npz(name='data/model' + support_name + '.npz')
tl.files.assign_weights(weights, train_model)

answers = []
qs = test_set['question'].to_numpy().tolist()

for test_query in qs:
    support_answer = predict2(test_query)
    answers.append(' '.join(support_answer))

test_result = pd.DataFrame({'question': qs, 'answer': answers})
test_result.to_csv(r'data/test' + support_name + '.csv')